In [1]:
import requests
import numpy as np
import pandas as pd

### Download the data

In [2]:
def df_from_api_url(url: str) -> pd.DataFrame:
    """Convert data.boston.gov's JSON API response into a dataframe."""
    json = requests.get(url).json()
    records = json["result"]["records"]
    df = pd.DataFrame(records).astype(str)
    return df


def make_url(resource_id: str) -> str:
    return f"https://data.boston.gov/api/3/action/datastore_search?resource_id={resource_id}&limit=100000"


# List table URLs by order of year: 2016, 2017, 2018, 2019
contact_urls = [
    make_url("35f3fb8f-4a01-4242-9758-f664e7ead125"),
    make_url("c72b9288-2658-4e6a-9686-ffdcacb585e7"),
    make_url("ee4f1175-54b6-4d06-bceb-26d349118e25"),
    make_url("35cfa498-cb10-43da-b8b2-948a66e48f26"),
    make_url("03f33240-47c1-46f2-87ae-bcdabec092ad"),
]
people_urls = [
    make_url("ebb9c51c-6e9a-40a4-94d0-895de9bf47ad"),
    make_url("f18a0632-46ea-4032-9749-f5b50cf7b865"),
    make_url("aa46b3ad-1526-4551-9f0f-6dbdfbb429c0"),
    make_url("b102d3a4-8b44-443e-bc09-00c44974c3b1"),
    make_url("2d29a168-534b-47c4-977a-b8f4aaf2ea8c"),
]

# Load, join, and concatenate all dataframes
contact_df = pd.concat([df_from_api_url(url) for url in contact_urls], sort=False)
people_df = pd.concat([df_from_api_url(url) for url in people_urls], sort=False)

### Clean up the data

In [3]:
df_transforms = []


def register_transform(f: callable) -> callable:
    df_transforms.append(f)
    return f


def apply_transforms(
    contact_df: pd.DataFrame, people_df: pd.DataFrame
) -> (pd.DataFrame, pd.DataFrame):
    for t in df_transforms:
        contact_df, people_df = t(contact_df, people_df)
    return contact_df, people_df


@register_transform
def rename_frisk_columns(contact_df, people_df):
    """
    Combine the `frisked` and `searchperson` columns from the ___contacts___ table into one column called `fc_involved_frisk_or_search`,
    and disambiguate it from a related column in the ___people___ table by renaming the `frisk/search` column to `person_frisked_or_searched`.
    As noted on data.boston.gov, the `frisked` and `searchperson` columns (from the "New RMS" data system) indicate whether any of the 
    individuals stopped in a given field contact was frisked, whereas `frisk/search` (from the "Mark43" data system) indicates whether
    a particular person involved in a field contact was frisked.
    """
    combine_fields = (
        lambda row: "Y"
        if row.frisked == "Y" or row.searchperson == "Y"
        else row.frisked
    )
    combined_values = contact_df[["frisked", "searchperson"]].apply(
        combine_fields, axis=1
    )
    contact_df = contact_df.assign(fc_involved_frisk_or_search=combined_values)
    contact_df = contact_df.drop(columns=["frisked", "searchperson"])

    people_df = people_df.rename(columns={"frisk/search": "person_frisked_or_searched"})
    people_df.person_frisked_or_searched = people_df.person_frisked_or_searched.replace(
        {"0": "N", "1": "Y"}
    )

    # The newer system doesn't provide contact-level frisk info,
    # but we can infer whether a contact involved a frisk/search based on
    # whether the people involved in that contact were frisked.
    contacts_with_no_data = contact_df.fc_involved_frisk_or_search.isnull()
    contact_involved_frisk = people_df.groupby("fc_num").apply(
        lambda g: "Y"
        if (g.person_frisked_or_searched == "Y").any()
        else "N"
        if (g.person_frisked_or_searched == "N").all()
        else np.nan
    )
    contact_df.loc[contacts_with_no_data, "fc_involved_frisk_or_search"] = contact_df[
        contacts_with_no_data
    ].apply(lambda row: contact_involved_frisk.loc[row.fc_num], axis=1)

    return contact_df, people_df


@register_transform
def vehicle_info_cleanup(contact_df, people_df):
    """
    Clean up and reconcile vehicle-related field values from the ___contacts___ table. Across the different data systems, different text values
    were used to represent identical or overlapping concepts (e.g. "LT. BLUE" and "light blue", "Suv (sport Utility Vehicle)" 
    and "SUV or Utility Van"). Also, drop the `vehicle_style` column, which is basically a noisier version of the `vehicle_type` column.
    """
    contact_df.vehicle_type = contact_df.vehicle_type.replace(
        {
            "Scooter": "Motorcycle or Scooter",
            "Cargo Van": "SUV or Utility Van",
            "Suv (sport Utility Vehicle)": "SUV or Utility Van",
            "Passenger Van": "Bus or Passenger Van",
            "Bus/Passenger Van": "Bus or Passenger Van",
            "Passenger Car/ Automobile": "Passenger Car",
        }
    )
    contact_df.vehicle_color = (
        contact_df.vehicle_color.str.lower()
        .str.strip()
        .replace(
            {
                "bla": "black",
                "gra": "gray",
                "gre": "green",
                "lt. green": "light green",
                "lt. blue": "light blue",
            }
        )
    )

    def fix_year(year: str) -> str:
        if not year or year in ("None", "NULL"):
            return year
        num_year = float(year)
        if np.isnan(num_year):
            return year
        if num_year > 1900:
            return year
        if num_year > 99:
            # Exclude definite typos
            return "NULL"
        if num_year < 10:
            return f"200{year}"
        if num_year < 21:
            return f"20{year}"
        else:
            return f"19{year}"

    contact_df.vehicle_year = contact_df.vehicle_year.apply(fix_year)

    return contact_df, people_df


@register_transform
def contact_date_to_dt(contact_df, people_df):
    """
    Convert the `contact_date` column in the ___contacts___ table to datetime, 
    and drop a duplicate column from the ___people___ table.
    """
    contact_df.contact_date = pd.to_datetime(contact_df.contact_date)

    return contact_df, people_df


@register_transform
def clean_officer_and_supervisor_names(contact_df, people_df):
    """
    Remove extra whitespace from the `contact_officer_name` column in the ___contacts___ table 
    to reduce accidental duplication. Also, put officer's first names first.
    """
    observed_names = {}

    def clean_name(id: str, name: str) -> str:
        if id in observed_names:
            return observed_names[id]

        name = name.replace(".", "")
        split_name = [part.strip() for part in name.split(",")]
        clean_name = None
        if len(split_name) == 1:
            clean_name = split_name[0]
        elif len(split_name) <= 3:
            last_part, *first_part = split_name
            clean_name = f'{" ".join(first_part)} {last_part}'

        if not clean_name:
            raise Exception(f"Encountered name with unexpected structure: {name}")

        clean_name = clean_name.upper()
        observed_names[id] = clean_name
        return clean_name

    contact_df.supervisor_name = contact_df.apply(
        lambda row: clean_name(row.supervisor, row.supervisor_name), axis=1
    )
    contact_df.contact_officer_name = contact_df.apply(
        lambda row: clean_name(row.contact_officer, row.contact_officer_name), axis=1
    )
    return contact_df, people_df


@register_transform
def combine_contact_reason_and_narrative(contact_df, people_df):
    """
    Merge the ___contacts___ table's `contact_reason` column into the `narrative` column. `contact_reason` serves the same purpose as `narrative`,
    just for the older system. Uppercase both columns for consistency.
    """
    empty_narrative = contact_df.narrative.isnull()
    contact_df.loc[empty_narrative, "narrative"] = contact_df[
        empty_narrative
    ].contact_reason
    contact_df.narrative = contact_df.narrative.str.upper()
    contact_df = contact_df.drop(columns=["contact_reason"])
    return contact_df, people_df


@register_transform
def clean_up_city(contact_df, people_df):
    """
    There are lots of typos and inconsistencies in the ___contacts___ table's `city` column, so fix them.
    Warning: current solution is not robust to *new* typos, should the data change.
    """
    BOSTON = "BOSTON"
    SOUTHIE = "SOUTH BOSTON"
    DORCHESTER = "DORCHESTER"
    CHARLESTOWN = "CHARLESTOWN"
    JP = "JAMAICA PLAIN"
    EASTIE = "EAST BOSTON"
    MATTAPAN = "MATTAPAN"
    ROXBURY = "ROXBURY"
    HYDEPARK = "HYDE PARK"
    contact_df.city = contact_df.city.str.upper()
    contact_df.city = contact_df.city.replace(
        {
            "BSTN": BOSTON,
            "SO.BOSTON": SOUTHIE,
            "DORCCHESTER": DORCHESTER,
            "CHAARLESTOWN": CHARLESTOWN,
            "JP": JP,
            "EAST BOS": EASTIE,
            "CHALRESTOWN": CHARLESTOWN,
            "EAST BOSTN": EASTIE,
            "DOR": DORCHESTER,
            "MT": MATTAPAN,
            "S BOSTON": SOUTHIE,
            "DORCHSTER": DORCHESTER,
            "BST": BOSTON,
            "S BSTN": SOUTHIE,
            "DORCHESTERR": DORCHESTER,
            "JAMAIICA PLAIN": JP,
            "ROXBURY MA": ROXBURY,
            "SO BOSTON": SOUTHIE,
            "E. BOSTON": EASTIE,
            "JAMAICA": JP,
            "DDORCHESTER": DORCHESTER,
            "MATTPAN": MATTAPAN,
            "JAMAICIA": JP,
            "S. BOSTON": SOUTHIE,
            "HP": HYDEPARK,
            "DORCHEST": DORCHESTER,
            "SBOS": SOUTHIE,
            "ROX": ROXBURY,
            "CHARLESTWON": CHARLESTOWN,
            "JAMACIA PLAIN": JP,
            "ROBURY": ROXBURY,
            "BTSN": BOSTON,
            "SOMMERVILLE": "SOMERVILLE",
            "JAMAICIA PLAIN": JP,
            "S.BOSTON": SOUTHIE,
            "DOR.": DORCHESTER,
            "E BOSTON": EASTIE,
            "E.BOSTON": EASTIE,
            "BOSTOB": BOSTON,
            "ROSLINDLAE": "ROSLINDALE",
            "BSNT": BOSTON,
            "BSTNA": BOSTON,
            "BSTON": BOSTON,
            "JAMAIACA PLAIN": JP,
            "SO. BOSTON": SOUTHIE,
        }
    )
    return contact_df, people_df


@register_transform
def clean_up_age(contact_df, people_df):
    """Remove implausibly high values in the ___people___ table's `age` column. Also, convert string values to float."""
    people_df.age = people_df.age.apply(
        lambda age: np.nan if len(age) > 2 or age == "" else np.float(age)
    )
    return contact_df, people_df


@register_transform
def combine_skin_tone_and_complexion(contact_df, people_df):
    """Merge the ___people___ table's older `complexion` column into the newer `skin_tone` column."""
    empty_skin_tone = people_df.skin_tone.isnull()
    people_df.loc[empty_skin_tone, "skin_tone"] = people_df.complexion[empty_skin_tone]
    people_df.skin_tone = people_df.skin_tone.replace({"OTHER": "Other"})
    people_df = people_df.drop(columns=["complexion"])
    return contact_df, people_df


@register_transform
def drop_deceased_column(contact_df, people_df):
    """
    Drop the `deceased` column from the ___people___ table, since it doesn't exist in the older system, and since no one 
    is marked deceased in this dataset.
    """
    people_df = people_df.drop(columns=["deceased"])
    return contact_df, people_df


@register_transform
def reconcile_hair_style(contact_df, people_df):
    """Reconcile the ___people___ table's `hair_style` values that seem to mean the same thing."""
    people_df.hair_style = people_df.hair_style.replace(
        {
            "Receding / Thin": "Receding or Thin",
            "Receding or Slightly Receding": "Receding or Thin",
            "Bald": "Bald or Balding",
            "Braids": "Braided",
            "Wig/hair Piece": "Wig or Hair Piece",
        }
    )
    return contact_df, people_df


@register_transform
def reconcile_race(contact_df, people_df):
    """Reconcile ___people___ table `race` values that seem to mean the same thing."""
    people_df.race = people_df.race.replace(
        {"American Indian or Alaskan Native": "Native American / Alaskan Native"}
    )
    return contact_df, people_df


@register_transform
def clean_up_state(contact_df, people_df):
    """Fix recurrent typo in the ___contacts___ table `state` column."""
    contact_df.state = contact_df.state.replace({"MX": "MA"})
    return contact_df, people_df


@register_transform
def reconcile_stop_duration(contact_df, people_df):
    """Bucket ___contacts___ table `stop_duration` column values listed as minutes and remove likely typos."""

    def bucket_stop_duration(d: str) -> str:
        try:
            d = int(d)
        except:
            return d

        # A stop longer than 8 hours seems really implausible
        if d > 500:
            return "NULL"
        if d < 5:
            return "Less Than Five Minutes"
        if d < 10:
            return "Five to Ten Minutes"
        if d < 15:
            return "Ten to Fifteen Minutes"
        if d < 20:
            return "Fifteen to Twenty Minutes"
        if d < 25:
            return "Twenty to Twenty-Five Minutes"
        if d < 30:
            return "Twenty-Five to Thirty Minutes"
        if d < 45:
            return "Thirty to Forty-Five Minutes"
        if d < 60:
            return "Forty-Five to Sixty Minutes"
        if d < 120:
            return "One to Two Hours"
        return "Longer Than Two Hours"

    contact_df.stop_duration = contact_df.stop_duration.apply(bucket_stop_duration)
    return contact_df, people_df


@register_transform
def clean_location_info(contact_df, people_df):
    """Combine the ___contacts___ table's `street` and `streetaddr` columns into single `street` column, and trim extra digits from `zip`."""
    empty_street = contact_df.street.isnull()
    contact_df.loc[empty_street, "street"] = contact_df.streetaddr[empty_street]
    contact_df.street = contact_df.street.apply(lambda s: s.replace("&", "/"))
    contact_df.zip = contact_df.zip.apply(lambda zip: zip.split("-")[0])
    return contact_df, people_df


@register_transform
def clean_empty_values(contact_df, people_df) -> pd.DataFrame:
    """
    The values `''`, `'NULL'`, and `'None'` are used to signify no value was entered in various fields across the dataset.
    Replace these values with `np.nan`.
    """
    replacements = {"": np.nan, "NULL": np.nan, "None": np.nan}
    contact_df = contact_df.replace(replacements)
    people_df = people_df.replace(replacements)
    return contact_df, people_df


clean_contact_df, clean__people__df = apply_transforms(contact_df, people_df)

### Save the cleaned data as CSVs

In [4]:
clean_contact_df.to_csv("foi_contacts.csv", index=False)
clean__people__df.to_csv("foi_people.csv", index=False)

### Auto-populate the README with changes applied to the data

In [5]:
readme_changelist = "\n".join([f"- {f.__doc__.strip()}" for f in df_transforms])

with open("README.md", "r") as readme:
    readme_body = readme.read()
    readme_trunc_body, _ = readme_body.split("## Data-cleaning operations")

with open("README.md", "w") as readme:
    readme.write(
        f"{readme_trunc_body}\n## Data-cleaning operations\n{readme_changelist}"
    )